In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,31157
2,Huelva,Confirmados PDIA 14 días,1379
3,Huelva,Tasa PDIA 14 días,"268,72186604828806"
4,Huelva,Confirmados PDIA 7 días,697
5,Huelva,Total Confirmados,31337
6,Huelva,Curados,28883
7,Huelva,Fallecidos,383


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  31157.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  8252.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 549 personas en los últimos 7 días 

Un positivo PCR cada 314 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,31157.0,697.0,1379.0,513170.0,135.822437,268.721866,150.0
Huelva-Costa,17936.0,363.0,691.0,289548.0,125.367815,238.647824,74.0
Condado-Campiña,10129.0,275.0,572.0,156231.0,176.021404,366.124521,56.0
Huelva (capital),8252.0,262.0,458.0,143837.0,182.150629,318.415985,55.0
Sierra de Huelva-Andévalo Central,2746.0,58.0,112.0,67391.0,86.064905,166.194299,20.0
Almonte,1815.0,66.0,133.0,24507.0,269.310809,542.702085,20.0
Valverde del Camino,562.0,25.0,50.0,12750.0,196.078431,392.156863,10.0
Aljaraque,1085.0,32.0,59.0,21474.0,149.017416,274.750862,8.0
Rociana del Condado,554.0,10.0,20.0,7939.0,125.960448,251.920897,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,171.0,6.0,18.0,2211.0,271.370421,814.111262,3.0
Cabezas Rubias,37.0,1.0,5.0,706.0,141.643059,708.215297,0.0
Palma del Condado (La),1146.0,22.0,68.0,10801.0,203.684844,629.571336,4.0
Niebla,233.0,16.0,25.0,4116.0,388.726919,607.385811,1.0
Almonte,1815.0,66.0,133.0,24507.0,269.310809,542.702085,20.0
Calañas,190.0,2.0,13.0,2768.0,72.254335,469.653179,1.0
Moguer,1276.0,42.0,96.0,21867.0,192.070243,439.017698,5.0
Manzanilla,98.0,3.0,9.0,2118.0,141.643059,424.929178,0.0
Campofrío,10.0,1.0,3.0,713.0,140.252454,420.757363,NaN


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Calañas,190.0,2.0,13.0,2768.0,72.254335,469.653179,1.0,0.153846
Cabezas Rubias,37.0,1.0,5.0,706.0,141.643059,708.215297,0.0,0.200000
Punta Umbría,805.0,13.0,43.0,15355.0,84.662976,280.039075,2.0,0.302326
Palma del Condado (La),1146.0,22.0,68.0,10801.0,203.684844,629.571336,4.0,0.323529
Villarrasa,171.0,6.0,18.0,2211.0,271.370421,814.111262,3.0,0.333333
Manzanilla,98.0,3.0,9.0,2118.0,141.643059,424.929178,0.0,0.333333
Puebla de Guzmán,114.0,1.0,3.0,3092.0,32.341527,97.024580,1.0,0.333333
Campofrío,10.0,1.0,3.0,713.0,140.252454,420.757363,0.0,0.333333
Ayamonte,1317.0,7.0,20.0,21104.0,33.169067,94.768764,1.0,0.350000
